# 测试代码可行性

In [ ]:
import os

In [5]:
print(os.path.split("D://myCode/PYTHON_file/Surf/测试网络请求的图片识别")[0])
print(os.path.split("")[0])

D://myCode/PYTHON_file/Surf



In [2]:
import base64

def str2base64str(sentence):
    return base64.b64encode(sentence.encode('utf8')).decode('utf8')

def base64str2str(b64str):
    return base64.b64decode(b64str).decode('utf8')

print(base64str2str("QmlnRm9sZGVyXFNtYWxsRm9sZGVy"))

BigFolder\SmallFolder
